In [ ]:
#| default_exp calibration

# RedTideProcessing Calibration

> Calibrate a HABSpec for wavelength, dark background, and radiance. 

Calibrating the HABSpec is done is three steps: 
1. Background: While the HABSpec does not have to be calibrated for wavelength or radiance to work with background data,
it is best to calibrate for wavelength first. Calibrating for wavelength first, allows the calibration code
to discriminate and detect the spectral source based in part on the wavelength of each pixel. If wavelength is
not calibrated, the calibration software cannot differentiate between light sources.
Background data is the baseline digital spectra when input is completely dark, and no light is 
entering the HABSpec.  We need to measure this for each possible exposure setting so it can be subtracted from the
each spectra.  The background values are not trivial, and are on the order of 35,000 counts for each pixel and thus
must be known and subtracted from each spectra in order to be able to apply radiometric calibration to the HABSpec.

1. Wavelength: We can calibrate the wavelength vs X axis pixel number by using a Compact Fluorescent Light (CFL)
and/or we can use the Fraunhofer lines present in sunlight. To calibrate wavelength we simply have to identify
the pixel location of two peaks in the CFL spectra, or two pixel locations of absorption peaks (negative peaks)
in the sunlight spectra.

1. Radiance: To calibrate for radiance, the HABSpec must be calibrated for wavelength and background must be removed.
Once the wavelengths of each pixel is known and calibrated and the background levels for each exposure setting
are known, we can then proceed to calibrate the HABSpec for Radiance and also eliminate variations in signal amplitude
that occur due to the combination of the spectral responses of the sensor, the diffraction gratings, and internal lenses, and
other optical elements.


Calibration steps:
1. Collect data:
   1. Background: Cover the entrance and collect 30 seconds of spectra at each exposure.
   1. Wavelengths: Remove the cover, turn on your CFL, or setup the HABSpec to
      view solar spectra and adjust the exposure so the spectra is fully on scale, 
      and then collect at least a few spectra. Make sure there are no *saturated* pixels.
   1. Radiance: Configure the HABSpec to collect data from the radiance bulb and reflector panel. Position the bulb 0.5 meters from the reflector, and collect spectra at all exposures that don't cause saturated pixels.
   1. Radiance: Reposition the bulb to 1.0 meters from the reflectors, and repeat collecting spectra at difference exposures that don't cause saturated pixels.

1. Process the calibration data:
   1. Wavelengths:
   1. Background: 
   1. Radiance:
   
2. Save the Calibration:

## References
- [Opean Optics Web Book](https://www.oceanopticsbook.info/)
- [Light from the Sun](https://www.oceanopticsbook.info/view/light-and-radiometry/level-2/light-from-the-sun)

## Install

```sh
pip install RedTideProcessing
```

## How to use

Fill me in please! Don't forget code examples:

# Imports

In [ ]:
#| hide
from   nbdev.showdoc import *
import nbdev

In [ ]:
#| export
import getpass
import gzip
import numpy                                as np
import os
import datetime                             as dt
import pickle
import socket
import time

In [ ]:
#| export
import RedTideProcessing.habspec_data_class as hsdc
import RedTideProcessing.habspec            as habspec
import RedTideProcessing.wavelength_cal     as cal
import RedTideProcessing.testing            as testing
import RedTideProcessing.radiance_cal       as rad_cal

In [ ]:
import panel                                as pn
from bokeh.plotting                     import ColumnDataSource
from bokeh.plotting                     import figure, show 
from bokeh.io                           import show, output_notebook

In [ ]:
rv = output_notebook()

Loading BokehJS ...

# Local Functions

## mk_figure().
Pre configures Bokeh figure parameters with common defaults values for this application.

## def mk_figure()

In [ ]:
def mk_figure(       title = '',                  # Plot title.
              x_axis_label = 'Wavelengthₙₘ',   # X axis label.
              y_axis_label = 'Counts',            # Y axis label.
                    height = 400,                 # Height of plot in pixels.
                     width = 700,                 # Width of plot i npixels.
                     tools ='pan, hover, wheel_zoom, box_zoom, undo, redo, reset', # The default set of plot tools.
             active_scroll = 'wheel_zoom',
            output_backend ="webgl",                # Use webGL enhancement as default.
                   **kwargs
             )-> object:                          # A Bokeh figure object.
  '''
  Calls bokeh figure with settings preset to more likly default values. 
  '''  
  rv = figure(  title = title,
              x_axis_label = x_axis_label,
              y_axis_label = y_axis_label,
                    height = height,
                     width = width,
                     tools = tools,
             active_scroll = active_scroll,
            output_backend = output_backend,
              **kwargs
             )
  return rv

## def configure_legend( plot )

In [ ]:
def configure_legend( plot ):
  '''
  Configure a Bokeh plot legend.
  '''
  plot.legend.label_height         = 10
  plot.legend.glyph_height         = 10
  plot.legend.glyph_width          = 10
  plot.legend.spacing              = 0
  plot.legend.label_text_font_size = '7pt'
  plot.legend.location             = 'top_left'

## class dark_spec()

In [ ]:
class dark_spec():
  '''

  '''
  def __init__( self, exposure ):
    self.count    = 0
    self.exposure = exposure
    self.sum      = np.zeros(1280)

## def compute_key()

In [ ]:
def compute_key( spectra ):
  '''
  Compute the key value based on aspects of the spectra. Use to 
  automatically discriminate between CFL bulb spectra, Solar, 
  and dark background spectra.
  '''
  if spectra.raw_y[600] < 40000:                # Is it a dark background ?             
    key = f'{spectra.Exposure:0.0f}-background' # Create a background key
  else:
    key = f'{spectra.Exposure:0.0f}-QTH'        # Quartz Tungsten Halogen radiance cal spectra.
  return key

# Setup.

## hab_spectra

Create a HAB_Spectra instance to hold a spectra as we progress throught he calibration process.

In [ ]:
hab_spectra = hsdc.HAB_Spectra()

## cal_data class.

Create a calibration record instance to store calibration data as it is developed. 
This record will be saved to a file once calibration is completed.  The saved 
record will be used on the HABSpec for realtime calibration of data, and also
used for processing post flight.

In [ ]:
cal_data = hsdc.HABSPEC_CAL_DATA()
cal_data.create_date

datetime.datetime(2023, 8, 6, 13, 0, 19, 792031)

In [ ]:
cal_data.cfl_cal_fn          = "/mnt/s/2023-0805-HAB-cal/wavelength/141053/hab_spectra/2023-0805-141324-276614-spec.json"
cal_data.solar_cal_fn        = "/mnt/s/2023-0805-HAB-cal/wavelength/142014/hab_spectra/2023-0805-142024-739094-spec.json"
cal_data.radiance_cal_path   = "/mnt/s/2023-0805-HAB-cal/QTH-75W"
cal_data.dark_cal_path       = "/mnt/s/2023-0805-HAB-cal/dark"
cal_data.radiance_cal_src_fn = "../2023-0728-newport-6333-QTH.csv"

In [ ]:
#print(cal_data.__str__())

# Wavelength Calibration

## Wavelength Calbration using CFL light.

Paste the full path name of the calibration json spectra file below and assign to `wavelength_cal_file`.  The spectra will be loaded
into the `hab_spectra.raw_y`

In [ ]:
spec_y                = habspec.read_spectra(hab_spectra,  cal_data.cfl_cal_fn, y_average=False, remove_bias=False )
print(f'CFL Calibration File: {os.path.basename( cal_data.cfl_cal_fn ) }')

CFL Calibration File: 2023-0805-141324-276614-spec.json


Next, we will generate the normalized y values as `normalized_cfl_y` so the solar spectra can be
displayed on the same plot as the solar spectra data.

In [ ]:
normalized_cfl_y = (hab_spectra.raw_y - hab_spectra.raw_y_min) / hab_spectra.Pix_maxv
cfl_source = ColumnDataSource( data=dict(
   x = hab_spectra.Xpixels,
   y = normalized_cfl_y )
)

## Wavelength Calibration Using Solar Spectrum.

If you are using a solar spectrum to calibrate wavelength, Paste the full path name of the solar calibration json spectra file below and assign to `wavelength_cal_file`.  The spectra will be loaded
into the `hab_spectra.raw_y`

In [ ]:
spec_y                = habspec.read_spectra(hab_spectra,  cal_data.solar_cal_fn, y_average=False, remove_bias=False )
print(f'Solar Calibration File: {os.path.basename( cal_data.solar_cal_fn ) }')

Solar Calibration File: 2023-0805-142024-739094-spec.json


Now, we will generate the normalized y values for the solar spectra as `normalized_solar_y`.
This is to insure the solar spectra can be displayed on the same plot as the CFL data.

In [ ]:
normalized_solar_y    = (hab_spectra.raw_y - hab_spectra.raw_y_min) / hab_spectra.Pix_maxv
solar_source = ColumnDataSource( data=dict(
   x = hab_spectra.Xpixels,
   y = normalized_solar_y )
)

## Plot Wavelength Calibration Sources vs Pixel

Now we configure Bokeh to plot the `hab_spectra.raw_y` spectra data vs `hab_spectra.Xpixels`.  This will allow us
to view the spectra and to identify known spectral peaks and determine what pixel the peak occurs in.

In [ ]:
TOOLTIPS = [ ("Pixel Number:", "$index"), ("Normalized Intensity:", "@y{0.000}") ]
wavelength_w = mk_figure(      #title = f"File: { title }",
                     x_axis_label = 'Pixel Number', 
                      y_axis_label= 'Normalized Intensity',
                         tooltips = TOOLTIPS,
                      #active_scroll = 'wheel_zoom',
                  )

if 'normalized_cfl_y' in globals() :
  cfl_color = 'blue'
  wavelength_w.title.text = "  CFL File: "+os.path.basename(cal_data.cfl_cal_fn)
  junk = wavelength_w.line(   'x', 'y', source=cfl_source, legend_label = f"CFL", line_width = 1, color=cfl_color )
  junk = wavelength_w.circle( 'x', 'y', source=cfl_source, legend_label = f"CFL", line_width = 1, color=cfl_color )
  
if 'normalized_solar_y' in globals() :
  solar_color = 'orange'
  wavelength_w.title.text += "\n"+"Solar File: "+os.path.basename(cal_data.solar_cal_fn)
  junk = wavelength_w.line(   'x', 'y', source=solar_source, legend_label = f"Solar", line_width = 1, color=solar_color )
  junk = wavelength_w.circle( 'x', 'y', source=solar_source, legend_label = f"Solar", line_width = 1, color=solar_color )

wavelength_w.legend.location = 'top_left'

show(wavelength_w)

## Determine Calibration Pixels and Wavelengths

Now that we can view the wavelength calibration spectra graphically above, we will now
examine the spectra and find peaks and valleys of known wavelengths in either the 
CFL, or solar, or both spectra. We will find the X-axis pixel values that coorespond
to the wavelengths and assign to variables as follows:
- `pixel0` will coorespond to `wavelength0`.  These should be the left most positively
identifiable peak or valley.
- `pixel1` and `wavelength1` should be a peak or valley as far to the right side
as possible.

The `cal` module contains specific wavelengths for *Fraunhofer solar spectral lines* as
well as the *Mecury (HG) spectral lines* that are present in CFL light bulbs.  You can access
and assign wavelengths from these Python `dictionaries` as shown below, or of you know
the wavelengths, you can simply assign the wavelengths yourself.

Below are two examples.

In [ ]:
cal.Fraunhofer_lines['A'], cal.HG_lines['Hg-436']

(['O2', 759.37], ['Hg', 435.8328])

Below we will assign `pixel0` `pixel1` and `wavelength0` and `wavelength1` based on peaks and valleys
we found in our calibration spectra from both a CFL light and the sun.

In [ ]:
  pixel0 =  91; wavelength0 = cal.HG_lines['Hg-436'][1]
  pixel1 = 943; wavelength1 = cal.Fraunhofer_lines['A'][1] 

Now we can call `habspec.calibrate_using_2_wavelengths()` which will set the calibration for all `hab_spectra`.
The calibration parameters internal to `hab_spectra` are *class variables* and are thus shared with all instances
of `hab_spectra`.

In [ ]:
rv = habspec.calibrate_using_2_wavelengths(   
  hab_spectra,
  cal_data.solar_cal_fn,
  pixel0 =  pixel0, wavelength0 =wavelength0, 
  pixel1 = pixel1, wavelength1 = wavelength1
)

Now we save the wavelength cal values to the calibration data class `cal_data`

In [ ]:
cal_data.pixel0 = pixel0; cal_data.wavelength0 = wavelength0
cal_data.pixel1 = pixel1; cal_data.wavelength1 = wavelength1

In [ ]:
hab_spectra.wavelengths

array([401.27659671, 401.65633521, 402.03607371, ..., 886.20265822,
       886.58239671, 886.96213521])

In [ ]:
print(cal_data.__str__())

create_date         : 2023-08-06 13:00:19.792031
Notes               : None
dark_cal_path       : /mnt/s/2023-0805-HAB-cal/dark
solar_cal_fn        : /mnt/s/2023-0805-HAB-cal/wavelength/142014/hab_spectra/2023-0805-142024-739094-spec.json
cfl_cal_fn          : /mnt/s/2023-0805-HAB-cal/wavelength/141053/hab_spectra/2023-0805-141324-276614-spec.json
radiance_cal_path   : /mnt/s/2023-0805-HAB-cal/QTH-75W
radiance_cal_src_fn : ../2023-0728-newport-6333-QTH.csv
pixel0              : 91
pixel1              : 943
wavelength0         : 435.8328
wavelength1         : 759.37
scale               : {1: [None], 2: [None], 3: [None], 4: [None], 6: [None], 7: [None], 8: [None], 9: [None], 10: [None], 15: [None], 20: [None], 25: [None], 30: [None], 35: [None], 50: [None], 75: [None], 100: [None], 150: [None], 200: [None], 250: [None], 300: [None], 350: [None], 400: [None], 500: [None], 750: [None], 1000: [None]}
dark                : {1: [None], 2: [None], 3: [None], 4: [None], 6: [None], 7: [None], 8

## Plot Wavelength Calibrated Spectra.

After completing the wavelength calibration above, we can now plot our calibration spectra vs
wavelength to verify other peaks and valleys are correct.

In [ ]:
# Generate the Bokeh plot widget.

TOOLTIPS = [ ("Pixel Number:", "$index"), ("Wavelength:","@x"), ("raw_y:", "@y") ]
spectra_w = mk_figure(      title = f"File: ",
                     x_axis_label = 'Wavelengthₙₘ',
                      y_axis_label= 'Normalized Y values',
                         tooltips = TOOLTIPS,
                     )

Generate and plot Fraunhofer lines.

In [ ]:
fran_x, fran_y = cal.gen_reference_lines( cal.Fraunhofer_lines, y_max=1.0 )
fran_source = ColumnDataSource( data=dict( x = fran_x,  y = fran_y ) )
junk = spectra_w.multi_line('x', 'y', source=fran_source, color='#ff000020', width=2, legend_label="Fraunhofer Lines." )


Generate and plot HG lines.

In [ ]:
hg_x, hg_y = cal.gen_reference_lines( cal.HG_lines, y_max=1.0 )
hg_source = ColumnDataSource( data=dict( x = hg_x,  y = hg_y ) )
junk = spectra_w.multi_line('x', 'y', source=hg_source, color='#0000ff20', width=2, legend_label="HG Lines." )

Plot the CFL and solar spectra.

In [ ]:
if 'normalized_cfl_y' in globals() :
  spectra_w.title.text = "  CFL File: "+os.path.basename(cal_data.cfl_cal_fn)
  cfl_color = 'blue'
  cfl_source.data['x'] = hab_spectra.wavelengths
  junk = spectra_w.line(   'x', 'y', source=cfl_source, legend_label = f"CFL", line_width = 1, color=cfl_color )
  junk = spectra_w.circle( 'x', 'y', source=cfl_source, legend_label = f"CFL", line_width = 1 )
  
if 'normalized_solar_y' in globals() :
  spectra_w.title.text += "\n"+"Solar File: "+os.path.basename(cal_data.solar_cal_fn)
  solar_color = 'orange'
  solar_source.data['x'] = hab_spectra.wavelengths
  junk = spectra_w.line(   'x', 'y', source=solar_source, legend_label = f"Solar", line_width = 1, color=solar_color )
  junk = spectra_w.circle( 'x', 'y', source=solar_source, legend_label = f"Solar", line_width = 1, color=solar_color )

spectra_w.legend.location = 'top_right'

show(spectra_w)

Now we will printout the variables stored in the class `hab_spectra` now.  Note that `pixel0`, `pixel1`, `wavelength0`, and `wavelength1` are all stored in the class now.

In [ ]:
print(hab_spectra.__str__())

self.Calibrated_w=True
self.Xpixels=array([   0,    1,    2, ..., 1277, 1278, 1279])
self.wavelengths=array([401.27659671, 401.65633521, 402.03607371, ..., 886.20265822,
       886.58239671, 886.96213521])
self.create_time=datetime.datetime(2023, 8, 6, 13, 0, 18, 437740)
self.pixel0=91
self.wavelength0=435.8328
self.pixel1=943
self.wavelength1=759.37
json_spectra_file_name: /mnt/s/2023-0805-HAB-cal/wavelength/142014/hab_spectra/2023-0805-142024-739094-spec.json
Lat                 : 28.2097385
Lon                 : -82.3639535
altitde_m           : None
n_saturated         : None
raw_y_min           : 36608
remove_bias         : False
y_average           : False
DateTime            : 2023-0805-142024.739
Exposure            : 10.0
raw_y               : [36800 36800 36800 ... 36752 36752 36752]
summed_rows         : 800
GPS_Alt             : 23.4
GPS_Nsat_Inview     : 13
GPS_Status          : 3
GPS_UTC             : 23:57:45
Source              : Water
Pix_minv            : 36608
sat_pi

# Background Calibration.

## Configure background source spectra data files.

In [ ]:
background_file_list   = habspec.get_list_of_json_spectra( cal_data.dark_cal_path )

Calibrate the `hab_spectra` by using the first spectra file found in the `background_file_list`. This
is done to cause the `hab_spectra` to get initilized with the correct array lengths that coorespond to
the actual sensor configuration in use.  Normally it is 1280 pixels wide and 800 pixels high, but it
is configurable and it is possilbe to be configured otherwise.

While wavelength is not strickly required to develop background data, it is needed to screen the spectra
to make sure they are in fact background, and do not contain any signals.  To discriminate the various
signals, the wavelength of each pixel is required.

## Iterate over the background files and compute a total sum of values for each pixel.

Next we read through all of the spectra in the `background_file_list`, examine the values at a
few pixels to determine if the spectra is of a dark background, a CFL calibration bulb, or a
Quarth Tungsten Halogen (QTH) radiance calibration bulb. A list `dark_list` is developed that contains
the sum of all the similar spectra for each exposure value.  The `dark_list` key is computed
when the spectra is read, and consists of a string consisting of the exposure value followed by 
the detected spectra type of either background, QTH, or Solar.  Only the background spectra
can be used as dark background.  The `compute_key()` is used to determine the type of data
in the spectra.

In [ ]:
last_exposure = 0
last_key      = ''
dark_list     = {}
print(f'Processing background spectra from: {cal_data.dark_cal_path}')
print('Exposure(ms)             key          raw_y[600]')
for fn in background_file_list:
  spec_y = habspec.read_spectra(hab_spectra,  fn, y_average=False, remove_bias=False )
  if compute_key( hab_spectra ) == last_key:
    d_spec.count += 1
    d_spec.sum   += spec_y
  else:
    d_spec = dark_spec( hab_spectra.Exposure )        # Create a new spectra record and key
    key = compute_key( hab_spectra )
    dark_list[ key ] = d_spec
    print(f'{d_spec.exposure:6}            {key:16s}     {hab_spectra.raw_y[600]:8.0f}')
    last_key = key
print('Done')

Processing background spectra from: /mnt/s/2023-0805-HAB-cal/dark
Exposure(ms)             key          raw_y[600]
2000.0            2000-background         36800
1000.0            1000-background         36800
 500.0            500-background          36800
 400.0            400-background          36800
 300.0            300-background          36800
 250.0            250-background          36776
 200.0            200-background          36728
 150.0            150-background          36704
 100.0            100-background          36632
  75.0            75-background           36464
  50.0            50-background           36608
  35.0            35-background           36584
  30.0            30-background           36512
  25.0            25-background           36560
  20.0            20-background           36512
  15.0            15-background           36512
  10.0            10-background           36392
   9.0            9-background            36416
   8.0            8-b

## Compute the average values for each spectra at each exposure value.  

Put the results in the `dark_list[exp].avg`.  

In [ ]:
# Compute the average values
print('# Exposure_ms                   Spectra  Average')
print('#     key             Exposure   Count    Value[600]')
for exp in dark_list.keys():
  dark_list[exp].avg = dark_list[exp].sum /  dark_list[exp].count
  print(f'{exp:16s}  {dark_list[exp].exposure:10.0f}       {dark_list[exp].count}     {dark_list[exp].avg[600]:5.0f}')

print('Done')

# Exposure_ms                   Spectra  Average
#     key             Exposure   Count    Value[600]
2000-background         2000       36     36801
1000-background         1000       34     36802
500-background           500       61     36800
400-background           400       50     36800
300-background           300       56     36796
250-background           250       54     36784
200-background           200       71     36755
150-background           150       88     36694
100-background           100       65     36688
75-background             75       50     36645
50-background             50       52     36571
35-background             35       52     36509
30-background             30       50     36526
25-background             25       51     36520
20-background             20       57     36491
15-background             15       52     36493
10-background             10       54     36467
9-background               9       52     36458
8-background               8      

## Store averaged background / dark data in `cal_data.dark`.

Copy the new average backgrounds to the `cal_data.dark[k]` data class so they can be saved and used with the 
system or for later processing.

In [ ]:
for exp in dark_list.keys():
  if exp.find("background") != -1:
    print(exp)
    k = int(dark_list[exp].exposure)
    cal_data.dark[k] = dark_list[exp].avg           # Fill in the dark offset values to the cal_data. 
    
print('Done')

2000-background
1000-background
500-background
400-background
300-background
250-background
200-background
150-background
100-background
75-background
50-background
35-background
30-background
25-background
20-background
15-background
10-background
9-background
8-background
7-background
6-background
5-background
4-background
3-background
2-background
1-background
Done


Show the status of the cal_data set as we build it.  Now it should have the dark values added.

In [ ]:
#print(cal_data.__str__())

## Plot all background / dark data in cal_data

In [ ]:
# Generate the Bokeh plot widget.
TOOLTIPS = [ ("Pixel Number:", "$index"), ("Wavelength:","@x{f0.1} nm"), ("Digital Counts:", "@y{f,}") ]
background_w = mk_figure(       title  = f"Background.",
                   x_axis_label  = 'Wavelengthₙₘ', 
                   y_axis_label  = 'Raw Digital Counts.',
                         height  = 700, 
                          width  = 700,
                        tooltips = TOOLTIPS,
                   output_backend="webgl"
                  )

# Plot the actual spectra
for i in dark_list:
  junk = background_w.line( hab_spectra.wavelengths, dark_list[i].avg, legend_label = f"{dark_list[i].exposure}", line_width = 1 )
#junk = background_w.line( hab_spectra.wavelengths, y, legend_label = f"{dark_list[i].exposure}", line_width = 1 )

configure_legend( background_w )

background_w.left[0].formatter.use_scientific = False
show(background_w)

# Radiance Calibration.

Set `cal_data.radiance_cal_src_fn` to the filename of the Quartz Tungsten Halogen calibration
file.  The file is a simple csv with columns for `wavelength`,`Irradiance`.  It is
read and then interpolated to the wavelengths of the HABSpec. 

Set `radiance_cal_path` to the 
directory where HABSpect json spectra files are located. The spectra files should contain 
spectra at all exposures with close to full scale signals for each one.

In [ ]:
radiance_file_list        = habspec.get_list_of_json_spectra( cal_data.radiance_cal_path )
rad_6333_irradiance_watts = rad_cal.read_radiance_calibration_file( hab_spectra.wavelengths, cal_data.radiance_cal_src_fn)
print( f'       radiance_cal_src_fn: {cal_data.radiance_cal_src_fn}')
print( f'cal_data.radiance_cal_path: {cal_data.radiance_cal_path}')

       radiance_cal_src_fn: ../2023-0728-newport-6333-QTH.csv
cal_data.radiance_cal_path: /mnt/s/2023-0805-HAB-cal/QTH-75W


## Process the radiance files.

In [ ]:
last_exposure     = 0
last_key          = ''
radiance_list     = {}
print(f'Processing background spectra from: {cal_data.radiance_cal_path}')
print('Exposure(ms)             key          raw_y[600]')
for fn in radiance_file_list:
  spec_y = habspec.read_spectra(hab_spectra,  fn, y_average=False, remove_bias=False )
  if compute_key( hab_spectra ) == last_key:
    d_spec.count += 1
    d_spec.sum   += spec_y
  else:
    d_spec = dark_spec( hab_spectra.Exposure )        # Create a new spectra record and key
    key = compute_key( hab_spectra )
    radiance_list[ key ] = d_spec
    print(f'{d_spec.exposure:6}            {key:16s}     {hab_spectra.raw_y[600]:8.0f}')
    last_key = key
print('Done')

Processing background spectra from: /mnt/s/2023-0805-HAB-cal/QTH-75W
Exposure(ms)             key          raw_y[600]
  35.0            35-QTH                 117994
  30.0            30-QTH                 107129
  25.0            25-QTH                  95048
  20.0            20-QTH                  85232
  15.0            15-QTH                  73459
  10.0            10-QTH                  63570
   9.0            9-QTH                   61442
   8.0            8-QTH                   59350
   7.0            7-QTH                   57048
   6.0            6-QTH                   55467
   5.0            5-QTH                   52968
   4.0            4-QTH                   51207
   3.0            3-QTH                   49272
   2.0            2-QTH                   47737
   1.0            1-QTH                   45643
Done


## Compute average spectra.

In [ ]:
print('# Compute the average values')
print('# Exposure_ms                     Spectra')
print('#    key             Exposure     Count')
for exp in radiance_list.keys():
  print(f'{exp:16s}  {radiance_list[exp].exposure:10.0f}       {radiance_list[exp].count}')
  radiance_list[exp].avg = radiance_list[exp].sum /  radiance_list[exp].count

print('Done')

# Compute the average values
# Exposure_ms                     Spectra
#    key             Exposure     Count
35-QTH                    35       73
30-QTH                    30       53
25-QTH                    25       54
20-QTH                    20       55
15-QTH                    15       55
10-QTH                    10       52
9-QTH                      9       53
8-QTH                      8       53
7-QTH                      7       82
6-QTH                      6       50
5-QTH                      5       51
4-QTH                      4       52
3-QTH                      3       53
2-QTH                      2       54
1-QTH                      1       53
Done


## Compute the radiance scale factor and store in cal_data.

In [ ]:
cal_data.scale = {}

In [ ]:
# This code is in the works to generate the scale factors
print('# Computing radiance scale factors')
print('# Exposure     Scale')
for k in radiance_list:
  if k.find('QTH') >=0:
    exp   = radiance_list[k].exposure
    dark  = cal_data.dark[exp]
    raw_y = radiance_list[k].avg   
    y_scale = rad_6333_irradiance_watts / (raw_y - dark)
    print(f'   {exp:4.0f}       {y_scale[600]:9.6e}')
    cal_data.scale[exp] = y_scale
    #print(exp, radiance_list[k].avg[600], cal_data.dark[exp][600] )
  else:
    #print(f'  Skipping.  {k} is not a QTH spectra.')
    pass
print("Done")

# Computing radiance scale factors
# Exposure     Scale
     35       3.004619e-04
     30       3.459187e-04
     25       4.145216e-04
     20       5.050823e-04
     15       6.490047e-04
     10       9.098250e-04
      9       9.823387e-04
      8       1.069921e-03
      7       1.171304e-03
      6       1.303100e-03
      5       1.462110e-03
      4       1.651581e-03
      3       1.904151e-03
      2       2.222265e-03
      1       2.724467e-03
Done


In [ ]:
#print(cal_data.__str__())

In [ ]:
# Generate the Bokeh plot widget.
TOOLTIPS = [ ("Pixel Number:", "$index"), ("Wavelength:","@x{f0.1} nm"), ("Scale Factor:", "@y") ]
scale_w = mk_figure(    title  = f"Scale Factors vs Wavelength & Exposure.",
                   x_axis_label  = 'Wavelengthₙₘ', 
                   y_axis_label  = 'Scale Factor.',
                        tooltips = TOOLTIPS
                  )

# Plot the radiance scale factors
for exp in cal_data.scale:
  junk = scale_w.line( hab_spectra.wavelengths, cal_data.scale[exp], legend_label = f"{exp}", line_width = 1 )

configure_legend( scale_w )
scale_w.legend.title = "Exposures (ms)"

scale_w.legend.location = 'top_center'
show(scale_w)

# Save Calibration Data.

In [ ]:
#print(cal_data.__str__())

In [ ]:
cal_data.Notes = "This is a test."

In [ ]:
#| export
def cal_verify( 
  cal           # Calibration data class.
) -> tuple:     # ( bool, list )
  '''
  Checks a calibration data class for missing items. Returns a tuple where the first
  element is either True or False, True returned if nothing missing, and False if there is a problem.
  The second element is a list of issues found with the cal object.
  '''
  rv = 0
  issue_list = []
  
  # Check for missing elements.
  for k in cal_data.__dict__.keys():
    if cal_data.__dict__[k] == None:
      issue_list.append(f'Error. {k} has no value')
      rv = (False, issue_list)
  
  # Get number of dark values
  dark_count = len(cal.__dict__['dark'])
  
  # Get number of scale values
  scale_count = len(cal.__dict__['scale'])
  
  # Check to make sure each scale factor exposure also has a dark value.
  dark = cal.__dict__['dark']
  for v in cal.__dict__['scale']:
    v = int(v)
    if v not in dark.keys():
      issue_list.append(f"Error. No dark value for exposure: {v} ")
      rv = (False, issue_list )
    #print(v, (v in dark.keys()) )
  
  if dark_count != scale_count:
    issue_list.append(f"Warning. Scale and Dark count mismatch. {scale_count} scale values and {dark_count} values found.")
    rv = (False, issue_list )
  
  return rv

In [ ]:
cal_verify( cal_data )

(False,
 ['Warning. Scale and Dark count mismatch. 15 scale values and 28 values found.'])

## Pickel cal data to a file.

In [ ]:
hostname = socket.gethostname()        # Include hostname where cal was done in the cal filename
username = getpass.getuser()           # Include the name of the user that ran the cal in the cal filename
cal_fn = f'{cal_data.create_date:%Y-%m%d-%H%M%S}-{hostname}-{username}-HABSpec-cal.pickel.gz'

In [ ]:
# Add code pre-check cal_data before writing.
p_odf = gzip.open( f'/tmp/{cal_fn}', 'wb')
pickle.dump( cal_data, p_odf)
p_odf.close()
print('Done')

Done


# Test calibration against indiv. QTH spectra.

Configure a list of json spectra files from each calibration QTH run.  We will apply the 
calibration values to each.

In [ ]:
path = "/mnt/s/2023-0805-HAB-cal/QTH-75W/"                   # Common path for test files.
test_spectra_files = [
  "144545/hab_spectra/2023-0805-144610-068363-spec.json",
  "144700/hab_spectra/2023-0805-144718-049499-spec.json",
  "144813/hab_spectra/2023-0805-144832-232449-spec.json",
  "144907/hab_spectra/2023-0805-144926-974476-spec.json",
  "145005/hab_spectra/2023-0805-145024-341413-spec.json",
  "145106/hab_spectra/2023-0805-145123-833036-spec.json",
  "145207/hab_spectra/2023-0805-145225-091188-spec.json",
  "145308/hab_spectra/2023-0805-145325-582449-spec.json",
  "145402/hab_spectra/2023-0805-145430-645003-spec.json",
  "145540/hab_spectra/2023-0805-145558-043705-spec.json",
  "145659/hab_spectra/2023-0805-145716-488524-spec.json",
  "145802/hab_spectra/2023-0805-145819-585688-spec.json",
  "145902/hab_spectra/2023-0805-145920-112706-spec.json",
  "150057/hab_spectra/2023-0805-150057-487990-spec.json",
  "145207/hab_spectra/2023-0805-145225-729969-spec.json",
  "145959/hab_spectra/2023-0805-150017-366226-spec.json",
]

In [ ]:
# Generate the Bokeh plot widget.
TOOLTIPS = [ ("Pixel Number", "$index"), ("Wavelengthₙₘ","@x{f0.1}"), ("Irradiance at 0.5m  mW m⁻² nm⁻¹", "@y{f0.00}") ]
full_cal_w = mk_figure(    title  = f"Fully Calibrated Spectra of QTH 6333 Lamp",
                   x_axis_label  = 'Wavelengthₙₘ', 
                   y_axis_label  = 'Irradiance at 0.5m  mW m⁻² nm⁻¹',                  
                        tooltips = TOOLTIPS
                  )
for s in test_spectra_files:
  y = habspec.read_spectra( hab_spectra, path+s, remove_bias=False, y_average=False)
  exp = hab_spectra.Exposure
  yc = rad_cal.radcal( hab_spectra, cal_data )
  junk = full_cal_w.line( hab_spectra.wavelengths, yc, legend_label = f"{exp}", line_width = 1 )
configure_legend( full_cal_w )
full_cal_w.legend.title = "Exposures (ms)"

full_cal_w.legend.location = 'top_left'
show(full_cal_w)